In [1]:
!pip install pandas 

In [2]:
import sqlite3
import pandas as pd
from datetime import datetime
from pathlib import Path
from IPython.display import display

# List of CSV files
csv_files = [
    Path(r"D:\BITS_SEM2\DMML_Assignment\Task4_DataPreparation\processed_data\clean_hf.csv"),
    Path(r"D:\BITS_SEM2\DMML_Assignment\Task4_DataPreparation\processed_data\clean_kaggle.csv")
]

# Columns that exist in the SQLite feature store table
table_cols = [
    "customerID", "tenure", "MonthlyCharges", "TotalCharges",
    "Contract_OneYear", "Contract_TwoYear",
    "PaymentMethod_CreditCard", "PaymentMethod_ElectronicCheck",
    "PaymentMethod_MailedCheck", "Churn"
]

# Use context manager to safely open/close the DB
with sqlite3.connect("feature_store.db") as conn:
    cursor = conn.cursor()

    # -------- Step 1: Create metadata table if not exists --------
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS feature_metadata (
        feature_name TEXT PRIMARY KEY,
        description TEXT,
        source TEXT,
        version TEXT,
        created_at TEXT
    )
    ''')

    # -------- Step 2: Create feature store table if not exists --------
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS feature_store (
        customerID TEXT PRIMARY KEY,
        tenure INTEGER,
        MonthlyCharges REAL,
        TotalCharges REAL,
        Contract_OneYear INTEGER,
        Contract_TwoYear INTEGER,
        PaymentMethod_CreditCard INTEGER,
        PaymentMethod_ElectronicCheck INTEGER,
        PaymentMethod_MailedCheck INTEGER,
        Churn INTEGER
    )
    ''')

    # -------- Step 3: Loop through CSV files --------
    for csv_file in csv_files:
        print(f"Processing: {csv_file.name}")
        df = pd.read_csv(csv_file)
        print(f"  Rows in file: {len(df)}")

        # Standardize column names: strip spaces, lowercase
        df.columns = df.columns.str.strip().str.replace(" ", "").str.lower()
        table_cols_lower = [col.lower() for col in table_cols]

        # Keep only columns that exist in both CSV and table
        df_to_insert = df[[col for col in table_cols_lower if col in df.columns]]

        # Rename columns to match SQLite table
        rename_map = dict(zip(df_to_insert.columns, [col for col in table_cols if col.lower() in df_to_insert.columns]))
        df_to_insert = df_to_insert.rename(columns=rename_map)

        # Ensure all table columns exist
        for col in table_cols:
            if col not in df_to_insert.columns:
                if col == "customerID":
                    df_to_insert[col] = ""
                elif col == "Churn":
                    df_to_insert[col] = 0
                else:
                    df_to_insert[col] = 0

        # Reorder columns to match table exactly
        df_to_insert = df_to_insert[table_cols]

        # Insert metadata dynamically
        features_meta = []
        for col in df_to_insert.columns:
            if col == "Churn":
                source = "Label"
                description = "Whether customer churned (0/1)"
            elif col.startswith("Contract") or col.startswith("PaymentMethod"):
                source = "Engineered"
                description = f"Feature: {col}"
            elif col == "customerID":
                source = "ID"
                description = "Unique customer identifier"
            else:
                source = "Telco DB"
                description = f"Feature: {col}"
            features_meta.append((col, description, source, "v1", str(datetime.now())))

        # Insert metadata safely
        try:
            cursor.executemany('''
            INSERT OR IGNORE INTO feature_metadata 
            (feature_name, description, source, version, created_at)
            VALUES (?, ?, ?, ?, ?)
            ''', features_meta)
        except sqlite3.OperationalError as e:
            print("Could not insert metadata:", e)

        # Insert data into the feature store
        records = df_to_insert.to_records(index=False)
        cursor.executemany('''
        INSERT OR REPLACE INTO feature_store VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', records)

    # -------- Step 4: Retrieve and display data --------
    features = pd.read_sql("SELECT * FROM feature_store", conn)
    print("Feature Store Data:")
    display(features)

    metadata = pd.read_sql("SELECT * FROM feature_metadata", conn)
    print("Feature Metadata:")
    display(metadata)


Processing: clean_hf.csv
  Rows in file: 1409
Processing: clean_kaggle.csv
  Rows in file: 7043


Feature Store Data:


,customerID,tenure,MonthlyCharges,TotalCharges,Contract_OneYear,Contract_TwoYear,PaymentMethod_CreditCard,PaymentMethod_ElectronicCheck,PaymentMethod_MailedCheck,Churn
0,C12345,24,55.2,1324.500000,1,0,1,0,0,0
1,C12346,12,70.0,840.000000,0,1,0,1,0,1
2,C12347,36,65.0,2340.000000,0,1,0,0,1,1
3,C12348,6,50.0,300.000000,1,0,1,0,0,0
4,C12349,18,60.0,1080.000000,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
8451,0.655144597170783,-0.340876,0.665992,-0.879397,b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',No
8452,-0.981733195159822,1.613701,1.277533,1.292099,b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',No
8453,-0.0757449459191446,-0.870241,-1.168632,-0.139680,b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',No
8454,1.18683476949932,-1.155283,0.320338,-0.316534,b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',b'\x00\x00\x00\x00\x00\x00\x00\x00',Yes


Feature Metadata:


,feature_name,description,source,version,created_at
0,tenure,Number of months the customer has stayed,Telco DB,v1,2025-08-24 12:40:09.769090
1,MonthlyCharges,Monthly fee charged to customer,Telco DB,v1,2025-08-24 12:40:09.769090
2,TotalCharges,Total amount charged,Telco DB,v1,2025-08-24 12:40:09.769090
3,Contract_OneYear,Contract type is one year,Engineered,v1,2025-08-24 12:40:09.769090
4,Contract_TwoYear,Contract type is two year,Engineered,v1,2025-08-24 12:40:09.769090
5,PaymentMethod_CreditCard,Payment via credit card,Engineered,v1,2025-08-24 12:40:09.769090
6,PaymentMethod_ElectronicCheck,Payment via electronic check,Engineered,v1,2025-08-24 12:40:09.769090
7,PaymentMethod_MailedCheck,Payment via mailed check,Engineered,v1,2025-08-24 12:40:09.769090
8,Churn,Whether customer churned (0/1),Label,v1,2025-08-24 12:40:09.769090
9,customerID,Unique customer identifier,ID,v1,2025-08-24 16:58:55.368119
